## Problem-2 part-2 Finetuning Indic BERT Model for NER classification:

In [1]:
!pip install transformers
!pip install torch
!pip install datasets
!pip install scikit-learn
!pip install accelerate

### Importing required libraries-

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset
import json
import time
from sklearn.metrics import f1_score
import numpy as np

2024-03-11 14:19:10.887079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 14:19:10.887228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 14:19:11.009461: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

### Loading dataset:

In [4]:
%%time
from datasets import load_dataset
# data = load_dataset('/content/drive/MyDrive/hi_IndicNER_v1.0')

from datasets import load_dataset

data = load_dataset('ai4bharat/naamapadam', 'hi')
print(data)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})
CPU times: user 3min 34s, sys: 2.97 s, total: 3min 37s
Wall time: 3min 41s


In [5]:
print(data['test'])

Dataset({
    features: ['tokens', 'ner_tags'],
    num_rows: 867
})


### Importing Tokenizers and Models from ai4bharat:

In [6]:
tokenizer_ner = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")
model_ner = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_ner = model_ner.to(device)

### Function to Tokenize data and assign corresponding labels with data:

In [8]:
padding = "max_length"
text_column_name = 'tokens'
label_column_name = 'ner_tags'

# Map NER labels to numerical values
label_mapping = {'O': 0, 'B-ORG': 1, 'I-ORG': 2, 'B-PER': 3, 'I-PER': 4, 'B-LOC': 5, 'I-LOC': 6}

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer_ner(
        examples[text_column_name],
        truncation=True,
        padding = padding,
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

### Selecting random data points to finetune the Model:

In [9]:
train_dataset1 = data['train'].shuffle(seed=42).select(range(50000))       #985787
validation_dataset1 = data['validation'] #13460
test_dataset1 = data['test']

### Mapping the data for compatibility with the Trainer class:

In [10]:
%%time
tokenized_train_dataset =train_dataset1.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_validation_dataset =validation_dataset1.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)
tokenized_test_dataset =test_dataset1.map(tokenize_and_align_labels,batched=True,load_from_cache_file=True,num_proc=10)

#0:   0%|          | 0/5 [00:00<?, ?ba/s]

#1:   0%|          | 0/5 [00:00<?, ?ba/s]

#2:   0%|          | 0/5 [00:00<?, ?ba/s]

#3:   0%|          | 0/5 [00:00<?, ?ba/s]

#4:   0%|          | 0/5 [00:00<?, ?ba/s]

...

#7:   0%|          | 0/1 [00:00<?, ?ba/s]

#6:   0%|          | 0/1 [00:00<?, ?ba/s]

#9:   0%|          | 0/1 [00:00<?, ?ba/s]

#8:   0%|          | 0/1 [00:00<?, ?ba/s]

CPU times: user 1.34 s, sys: 606 ms, total: 1.94 s
Wall time: 22.9 s


In [11]:
# print(tokenized_test_dataset)

In [12]:
# for token, label in zip(tokenizer_ner.convert_ids_to_tokens(tokenized_test_dataset["input_ids"][0]),tokenized_test_dataset["labels"][0]):
#     print(f"{token:_<40} {label}")

### Function to compute matric used to validate the performance of Model and Evaluate function to eval any dataset:

In [13]:
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(p):
    predictions, labels = p.predictions, p.label_ids

    # Flatten the predictions and labels
    predictions_flat = np.argmax(predictions, axis=2).flatten()
    labels_flat = labels.flatten()

    # Remove occurrences of -100 from both predictions and labels
    mask = (labels_flat != -100)
    predictions_filtered = predictions_flat[mask]
    labels_filtered = labels_flat[mask]

    # Calculate macro F1 score
    f1 = f1_score(labels_filtered, predictions_filtered, average="macro")

    # Calculate precision, recall, and accuracy with zero_division parameter
    precision = precision_score(labels_filtered, predictions_filtered, average="macro", zero_division=1)
    recall = recall_score(labels_filtered, predictions_filtered, average="macro")
    accuracy = accuracy_score(labels_filtered, predictions_filtered)

    return {
        "macro_f1": f1,
        "precision": precision,
        "recall": recall,
        "accuracy": accuracy
    }


def evaluate_model(trainer, dataset):
    results = trainer.evaluate(dataset)
    return results

### Specifing Training args and hyperparameters:

In [17]:
%%time
import numpy as np
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer_ner)

def fine_tune_indic_ner(train_dataset, validation_dataset):
    training_args = TrainingArguments(
        output_dir="./indic_ner_fine_tuned",
        num_train_epochs=5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir="./indic_ner_logs",
        evaluation_strategy="epoch",
        save_strategy = "epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
    )

    trainer = Trainer(
        model=model_ner,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    return trainer

CPU times: user 30 µs, sys: 1e+03 ns, total: 31 µs
Wall time: 36 µs


### Finetuning the Model with training data and val. data:

In [18]:
%%time
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

indic_ner_trainer = fine_tune_indic_ner(tokenized_train_dataset, tokenized_validation_dataset)

Epoch,Training Loss,Validation Loss,Macro F1,Precision,Recall,Accuracy
1,0.168900,0.181030,0.825597,0.842930,0.811202,0.946687
2,0.134600,0.181850,0.826177,0.832387,0.820943,0.945478
3,0.098800,0.203579,0.824514,0.825672,0.823902,0.944909
4,0.071900,0.232249,0.823776,0.833616,0.815444,0.944850
5,0.051800,0.253060,0.821829,0.830313,0.814370,0.944068


CPU times: user 4h 12min 41s, sys: 25.9 s, total: 4h 13min 7s
Wall time: 4h 12min 53s


### Model performance(macro f1-score) on test data:¶

In [21]:
indic_ner_results = evaluate_model(indic_ner_trainer, tokenized_test_dataset)
for metric in indic_ner_results:
    print(metric,":  ",indic_ner_results[metric])

eval_loss :   0.14882075786590576
eval_macro_f1 :   0.833918580479044
eval_precision :   0.8319968764341761
eval_recall :   0.841306851644088
eval_accuracy :   0.9531996179560649
eval_runtime :   16.981
eval_samples_per_second :   51.057
eval_steps_per_second :   3.239
epoch :   5.0


### Model performance(macro f1-score) on train data:¶

In [25]:
indic_ner_results = evaluate_model(indic_ner_trainer, tokenized_train_dataset)
for metric in indic_ner_results:
    print(metric,":  ",indic_ner_results[metric])

eval_loss :   0.08413787186145782
eval_macro_f1 :   0.9096231967722562
eval_precision :   0.9109521604281074
eval_recall :   0.9086517235905898
eval_accuracy :   0.9716792414053673
eval_runtime :   984.6087
eval_samples_per_second :   50.782
eval_steps_per_second :   3.174
epoch :   5.0


### Model performance(macro f1-score) on validation data:¶

In [26]:
indic_ner_results = evaluate_model(indic_ner_trainer, tokenized_validation_dataset)
for metric in indic_ner_results:
    print(metric,":  ",indic_ner_results[metric])

eval_loss :   0.18185046315193176
eval_macro_f1 :   0.8261773993050364
eval_precision :   0.8323873200649317
eval_recall :   0.8209433495394655
eval_accuracy :   0.9454780285328365
eval_runtime :   263.6154
eval_samples_per_second :   51.059
eval_steps_per_second :   3.194
epoch :   5.0


### To view logs of any epoch, uncomment below-

In [27]:
# train_metrics = indic_ner_trainer.state.log_history
# epoch_num = len(train_metrics)-1
# from pprint import pprint
# pprint(train_metrics[epoch_num])


## Saving Fine-tuned Model and tokenizer for furthur use in problem 4-

In [28]:
indic_ner_trainer.save_model("finetunedINDIC_ner_model")
tokenizer_ner.save_pretrained("finetunedINDIC_ner_model")

('finetunedINDIC_ner_model/tokenizer_config.json',
 'finetunedINDIC_ner_model/special_tokens_map.json',
 'finetunedINDIC_ner_model/vocab.txt',
 'finetunedINDIC_ner_model/added_tokens.json',
 'finetunedINDIC_ner_model/tokenizer.json')